In [1]:
import numpy as np
import os, shutil, glob, tempfile, gc
import math, random
from datetime import datetime
import PIL
import PIL.Image
import tensorflow as tf

from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

mos_img_dir = './datasets/Chula7MosDatasets/G-A52s_and_V-Y21/'
class_names = ['Ae-aegypti', 'Ae-albopictus', 'Ae-vexans', 'An-tessellatus', 'Cx-quinquefasciatus', 'Cx-vishnui', 'Misc']

#NUM_EPOCH_PER_TRAIN = 20
#no training

BATCH_SIZE = 8

# initial value of learning rate
#LR = 2e-5
# multiplier factor
#LR_MULTIPLIER = 0.5
# number of epochs per reduction
#NUM_EPOCH_PER_LR_UPDATE = 7

# original image size
IMG_SIZE = 224

# ratio of (train, val, test)
RATIO = (0.90, 0.10, 0.0) # train, val, test, (test all)

2022-10-27 02:06:38.457959: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def between0i1x(f):
    if 0 <= f < 1:
        return True
    else:
        return False
    

def RandomTrainValTest(src_dir, dst_dir, ratio = (0.8, 0.2, 0)):
    #check if Destination directory already exists
    if os.path.exists(dst_dir):
        print('Destination directory already exists. Do nothing!!!')
        return
    TRAIN_RATIO = round(ratio[0], 3)
    VAL_RATIO = round(ratio[1], 3)
    TEST_RATIO = round(1 - TRAIN_RATIO - VAL_RATIO, 3)
    if not between0i1x(TEST_RATIO) or not between0i1x(VAL_RATIO) or not between0i1x(TRAIN_RATIO):
        print('Ratio must be more than or equal 0 and less than 1.')
        return
    print('Ratio = (' + str(TRAIN_RATIO) +', ' + str(VAL_RATIO) + ', ' + str(TEST_RATIO) + ').' )
    if TEST_RATIO != round(ratio[2],3):
        print('Sum of ratio must be 1.')
        return
    
    # create a temp dir and copy all files recursively to temp_dir
    tmp = tempfile.TemporaryDirectory()
    tmp_dir = tmp.name + '/tmp/'
    shutil.copytree(src_dir, tmp_dir)
    # scan species, they must be immediatly under the src_dir 
    for species in os.scandir(src_dir):
        if not species.name.startswith('.') and species.is_dir():
            class_dir = tmp_dir + '/' + species.name
            print('Found ' + species.name)
            #calculate No. of files in each species 
            file_no = 0
            for entry2 in os.scandir(class_dir):
                if not entry2.name.startswith('.') and entry2.is_file():
                    file_no += 1
            print('No. of files in ' + species.name + ' = ' + str(file_no) + '.')
            # create subdirectories if necessary
            if not os.path.exists(dst_dir):
                os.mkdir(dst_dir)
            test_dir = dst_dir + '/test/' 
            val_dir = dst_dir + '/val/' 
            train_dir = dst_dir + '/train/' 
            if not os.path.exists(test_dir):
                os.mkdir(test_dir)
            if not os.path.exists(val_dir):
                os.mkdir(val_dir)
            if not os.path.exists(train_dir):
                os.mkdir(train_dir)
            # create species.name dir    
            stest_dir = test_dir + species.name + '/'
            sval_dir = val_dir + species.name + '/'
            strain_dir = train_dir + species.name + '/'
            if not os.path.exists(stest_dir):
                os.mkdir(stest_dir)
            if not os.path.exists(sval_dir):
                os.mkdir(sval_dir)
            if not os.path.exists(strain_dir):
                os.mkdir(strain_dir)
            # calculate number of files for train, val, test    
            no_file_train = math.ceil(file_no*TRAIN_RATIO)
            no_file_val = math.ceil(file_no*VAL_RATIO)
            no_file_test = math.ceil(file_no*TEST_RATIO)
            
            # for training data
            file_no = 0
            while file_no < no_file_train : 
                selectedFile = random.choice(os.listdir(tmp_dir + '/' + species.name))
                no = selectedFile.find('_s')
                no+=2
                while selectedFile[no:no+1].isnumeric():
                    no+=1                
                selectedFile = selectedFile[:no] + '*'
                #print('Selected ' + selectedFile)
                file_moved = 0
                for file in glob.iglob(class_dir +'/'+ selectedFile, recursive=True):    
                    file_name = os.path.basename(file)
                    shutil.move(file, strain_dir + file_name)
                    file_moved += 1
                # sum file_moved to file_no    
                file_no += file_moved
            print('No. of files in "' + strain_dir + '" should be around ' + str(no_file_train) + '.')
            print('No. of files in ' +strain_dir + ' = ' + str(file_no) +'.' )
            
            # for test data
            file_no = 0
            while file_no < no_file_test and os.listdir(tmp_dir + '/' + species.name): 
                selectedFile = random.choice(os.listdir(tmp_dir + '/' + species.name))
                #print(selectedFile)
                no = selectedFile.find('_s')
                no+=2
                while selectedFile[no:no+1].isnumeric():
                    no+=1
                selectedFile = selectedFile[:no] + '*'
                #print('Selected ' + selectedFile)
                file_moved = 0
                for file in glob.iglob(class_dir +'/'+ selectedFile, recursive=True):    
                    file_name = os.path.basename(file)
                    shutil.move(file, strain_dir + file_name)
                    file_moved += 1
                # sum file_moved to file_no    
                file_no += file_moved
            print('No. of files in "' + stest_dir + '" should be around ' + str(no_file_test) + '.')
            print('No. of files in ' +stest_dir + ' = ' + str(file_no) +'.' )
            #move all remaining files to val_dir
            selectedFile = '*'
            file_moved = 0
            for file in glob.iglob(class_dir +'/'+ selectedFile, recursive=True):    
                file_name = os.path.basename(file)
                shutil.move(file, sval_dir + file_name)
                file_moved += 1
            file_no = file_moved    
            print('No. of files in "' + sval_dir + '" should be around ' + str(no_file_val) + '.')
            print('No. of files in ' +sval_dir + ' = ' + str(file_no) +'.' )
                

def SplitLeftRight(src_dir, dst_dir):
    if not os.path.exists(src_dir):
        print('Source directory does not exist!')
        return
    if os.path.exists(dst_dir):
        print('Destination directory already exists.  Do nothing!!!')
        return
    trainS = src_dir + '/train/'    
    valS = src_dir + '/val/'
    testS = src_dir + '/test/'
    LEFT = '/Left/'
    RIGHT = '/Right/'
    src_subdir = ('train', 'val', 'test')

    if not os.path.exists(trainS) or not os.path.exists(valS) or not os.path.exists(testS):
        print('Source directory must contains all ["train", "val", "test"] subdirectories')
        return
    # copy source dir to tmp
    tmp = tempfile.TemporaryDirectory()
    tmp_dir = tmp.name + '/tmp/'
    shutil.copytree(src_dir, tmp_dir)

    # mkdir of destination structure
    os.mkdir(dst_dir)
    os.mkdir(dst_dir + LEFT)
    os.mkdir(dst_dir + RIGHT)
    
    # scan sub
    subidx = 0
    files_in_train_val_test = [0,0,0]
    for sub in src_subdir: # ('train', 'val', 'test')
        # create LEFT, RIGHT sub
        dst_left_sub = dst_dir + LEFT + sub
        dst_right_sub = dst_dir + RIGHT + sub
        os.mkdir(dst_left_sub)
        os.mkdir(dst_right_sub)
        # scan species
        for species in os.scandir(tmp_dir + sub):
            if not species.name.startswith('.') and species.is_dir():
                # count total files
                file_no = 0
                for entry2 in os.scandir(species):
                    if not entry2.name.startswith('.') and entry2.is_file():
                        file_no +=1
                print('There are ' + str(file_no)  + ' files in SRC_DIR/'+ sub + '/' + species.name +'.')
                # half of it
                half_file_no = math.floor(file_no/2)
                print('There should be ' + str(half_file_no)  + ' files in DST_DIR/[LEFT,RIGHT]/'+ sub + '/' + species.name +'.')
                print('.........')
                # get the file list
                FileList = sorted( filter( os.path.isfile, glob.glob(tmp_dir + sub + '/' + species.name + '/*') ) )
                # devide by half
                FileListL = FileList[0:half_file_no]
                FileListR = FileList[half_file_no:half_file_no*2]
                files_in_train_val_test[subidx] += half_file_no
                
                # create species directory
                dst_left_sub_species = dst_left_sub + '/' + species.name 
                dst_right_sub_species = dst_right_sub + '/' + species.name 
                os.mkdir(dst_left_sub_species)
                os.mkdir(dst_right_sub_species)
                for file in FileListL:
                    file_name = os.path.basename(file)
                    shutil.move(file, dst_left_sub_species + '/' + file_name)
                for file in FileListR:
                    file_name = os.path.basename(file)
                    shutil.move(file, dst_right_sub_species + '/' + file_name)
        subidx += 1
    print('There are ' +str(files_in_train_val_test[0]) + ' files in train, ' 
          + str(files_in_train_val_test[1]) + ' files in val ' 
          + str(files_in_train_val_test[2]) + ' files in test.')
    return (files_in_train_val_test[0], files_in_train_val_test[1], files_in_train_val_test[2])

def SplitLeftMiddleRight(src_dir, dst_dir):
    if not os.path.exists(src_dir):
        print('Source directory does not exist!')
        return
    if os.path.exists(dst_dir):
        print('Destination directory already exists.  Do nothing!!!')
        return
    trainS = src_dir + '/train/'    
    valS = src_dir + '/val/'
    testS = src_dir + '/test/'
    LEFT = '/Left/'
    MIDDLE = '/Middle/'
    RIGHT = '/Right/'
    src_subdir = ('train', 'val', 'test')

    if not os.path.exists(trainS) or not os.path.exists(valS) or not os.path.exists(testS):
        print('Source directory must contains all ["train", "val", "test"] subdirectories')
        return
    # copy source dir to tmp
    tmp = tempfile.TemporaryDirectory()
    tmp_dir = tmp.name + '/tmp/'
    shutil.copytree(src_dir, tmp_dir)

    # mkdir of destination structure
    os.mkdir(dst_dir)
    os.mkdir(dst_dir + LEFT)
    os.mkdir(dst_dir + MIDDLE)
    os.mkdir(dst_dir + RIGHT)
    
    # scan sub
    subidx = 0
    files_in_train_val_test = [0,0,0]
    for sub in src_subdir: # ('train', 'val', 'test')
        # create LEFT, RIGHT sub
        dst_left_sub = dst_dir + LEFT + sub
        dst_middle_sub = dst_dir + MIDDLE + sub
        dst_right_sub = dst_dir + RIGHT + sub
        os.mkdir(dst_left_sub)
        os.mkdir(dst_middle_sub)
        os.mkdir(dst_right_sub)
        # scan species
        for species in os.scandir(tmp_dir + sub):
            if not species.name.startswith('.') and species.is_dir():
                # count total files
                file_no = 0
                for entry2 in os.scandir(species):
                    if not entry2.name.startswith('.') and entry2.is_file():
                        file_no +=1
                print('There are ' + str(file_no)  + ' files in SRC_DIR/'+ sub + '/' + species.name +'.')
                # third of it
                third_file_no = math.floor(file_no/3)
                print('There should be ' + str(third_file_no)  + ' files in DST_DIR/[LEFT,MIDDLE,RIGHT]/'+ sub + '/' + species.name +'.')
                print('.........')
                # get the file list
                FileList = sorted( filter( os.path.isfile, glob.glob(tmp_dir + sub + '/' + species.name + '/*') ) )
                # devide by half
                FileListL = FileList[0:third_file_no]
                FileListM = FileList[third_file_no:third_file_no*2]
                FileListR = FileList[third_file_no*2:third_file_no*3]
                files_in_train_val_test[subidx] += third_file_no
                
                # create species directory
                dst_left_sub_species = dst_left_sub + '/' + species.name 
                dst_middle_sub_species = dst_middle_sub + '/' + species.name
                dst_right_sub_species = dst_right_sub + '/' + species.name 
                os.mkdir(dst_left_sub_species)
                os.mkdir(dst_middle_sub_species)
                os.mkdir(dst_right_sub_species)
                for file in FileListL:
                    file_name = os.path.basename(file)
                    shutil.move(file, dst_left_sub_species + '/' + file_name)
                for file in FileListM:
                    file_name = os.path.basename(file)
                    shutil.move(file, dst_middle_sub_species + '/' + file_name)
                for file in FileListR:
                    file_name = os.path.basename(file)
                    shutil.move(file, dst_right_sub_species + '/' + file_name)
        subidx += 1
    print('There are ' +str(files_in_train_val_test[0]) + ' files in train, ' 
          + str(files_in_train_val_test[1]) + ' files in val ' 
          + str(files_in_train_val_test[2]) + ' files in test.')
    return (files_in_train_val_test[0], files_in_train_val_test[1], files_in_train_val_test[2])
 

def randomSplitMosData(input_dir, output_dirLR, output_dirLMR, ratio = RATIO):
    tmpX = tempfile.TemporaryDirectory().name
    RandomTrainValTest(input_dir,tmpX, ratio = ratio)
    #tmpX = '/tmp/tmpk_mjnf4g'
    no_files_LR = SplitLeftRight(tmpX, output_dirLR)
    no_files_LMR = SplitLeftMiddleRight(tmpX, output_dirLMR)
    return no_files_LR, no_files_LMR    

In [3]:
def read_ds(src_dir):
    
    train_dir = src_dir  + '/Left/train'
    train_dir1 = src_dir + '/Right/train'
    val_dir = src_dir    + '/Left/val'
    val_dir1 = src_dir   + '/Right/val'
    
    train_ds = tf.keras.utils.image_dataset_from_directory(
      train_dir,
      #validation_split=0.2,
      #subset="training",
      seed=123,
      image_size=(IMG_SIZE, IMG_SIZE),
      shuffle = False,
      label_mode='categorical',
      batch_size=BATCH_SIZE)

    # val_ds = tf.keras.utils.image_dataset_from_directory(
    #   val_dir,
    #   #validation_split=0.2,
    #   #subset="validation",
    #   seed=123,
    #   shuffle = False,
    #   label_mode='categorical',
    #   image_size=(IMG_SIZE, IMG_SIZE),
    #   batch_size=BATCH_SIZE)



    train_ds1 = tf.keras.utils.image_dataset_from_directory(
      train_dir1,
      #validation_split=0.2,
      #subset="training",
      seed=123,
      shuffle = False,
      label_mode='categorical',
      image_size=(IMG_SIZE, IMG_SIZE),
      batch_size=BATCH_SIZE)
    
    val_ds = val_ds1 = 0

    # val_ds1 = tf.keras.utils.image_dataset_from_directory(
    #   val_dir1,
    #   #validation_split=0.2,
    #   #subset="validation",
    #   seed=123,
    #   shuffle = False,
    #   label_mode='categorical',
    #   image_size=(IMG_SIZE, IMG_SIZE),
    #   batch_size=BATCH_SIZE)
    
    return (train_ds, val_ds, train_ds1, val_ds1)

In [4]:
def combineDS(train_ds, val_ds, train_ds1, val_ds1):
    # Rescale all datasets
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), 
                    num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), 
    #                 num_parallel_calls=tf.data.AUTOTUNE)

    train_ds1 = train_ds1.map(lambda x, y: (normalization_layer(x), y), 
                    num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds1 = val_ds1.map(lambda x, y: (normalization_layer(x), y), 
    #                 num_parallel_calls=tf.data.AUTOTUNE)

    train_ds_X = train_ds.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    train_ds_X1 = train_ds1.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    train_ds_Y = train_ds.map(lambda x,y:(y), num_parallel_calls=tf.data.AUTOTUNE)


    # val_ds_X = val_ds.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds_X1 = val_ds1.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds_Y = val_ds.map(lambda x,y:(y), num_parallel_calls=tf.data.AUTOTUNE)
    
    # combine left, right 
    train_ds_X_X1 = tf.data.Dataset.zip((train_ds_X, train_ds_X1))


    # attach label back
    train_ds_X_X1_Y = tf.data.Dataset.zip((train_ds_X_X1, train_ds_Y))

    #--------------------------------------------------------------

    val_ds_X_X1_Y = 0
   
    
    return (train_ds_X_X1_Y, val_ds_X_X1_Y)


In [5]:
def read_dsLMR(LMR_dir):
    tmp_dir = LMR_dir
    tmp_dirL    = tmp_dir + '/Left/'
    tmp_dirM    = tmp_dir + '/Middle/'
    tmp_dirR    = tmp_dir + '/Right/'
    # for repeatable train, val, test folder spliting
    now = datetime.now()
    SEED = 123409 + np.random.randint(-100000,100000)

    train_dirL   = tmp_dirL + 'train'
    train_dirM  = tmp_dirM + 'train'
    train_dirR  = tmp_dirR + 'train'
    test_dirL    = tmp_dirL + 'val'
    test_dirM   = tmp_dirM + 'val'
    test_dirR   = tmp_dirR + 'val'

    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dirL,
        seed=SEED,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle = False,
        label_mode='categorical',
        crop_to_aspect_ratio = True,
        batch_size=BATCH_SIZE)


    # val_ds = tf.keras.utils.image_dataset_from_directory(
    #     test_dirL,
    #     seed=SEED,
    #     shuffle = False,
    #     label_mode='categorical',
    #     image_size=(IMG_SIZE, IMG_SIZE),
    #     batch_size=BATCH_SIZE)


    train_ds1 = tf.keras.utils.image_dataset_from_directory(
        train_dirM,
        seed=SEED,
        shuffle = False,
        label_mode='categorical',
        image_size=(IMG_SIZE, IMG_SIZE),
        crop_to_aspect_ratio = True,
        batch_size=BATCH_SIZE)


    # val_ds1 = tf.keras.utils.image_dataset_from_directory(
    #     test_dirM,
    #     seed=SEED,
    #     shuffle = False,
    #     label_mode='categorical',
    #     image_size=(IMG_SIZE, IMG_SIZE),
    #     batch_size=BATCH_SIZE)

    train_ds2 = tf.keras.utils.image_dataset_from_directory(
        train_dirR,
        seed=SEED,
        shuffle = False,
        label_mode='categorical',
        image_size=(IMG_SIZE, IMG_SIZE),
        crop_to_aspect_ratio = True,
        batch_size=BATCH_SIZE)


    # val_ds2 = tf.keras.utils.image_dataset_from_directory(
    #     test_dirR,
    #     seed=SEED,
    #     shuffle = False,
    #     label_mode='categorical',
    #     image_size=(IMG_SIZE, IMG_SIZE),
    #     batch_size=BATCH_SIZE)
    val_ds = val_ds1 = val_ds2 = 0
    return (train_ds, val_ds, train_ds1, val_ds1, train_ds2, val_ds2)

In [6]:
def combineDS_LMR(train_ds, val_ds, train_ds1, val_ds1, train_ds2, val_ds2):
    # Rescale all datasets
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), 
                    num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), 
    #                 num_parallel_calls=tf.data.AUTOTUNE)

    train_ds1 = train_ds1.map(lambda x, y: (normalization_layer(x), y), 
                    num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds1 = val_ds1.map(lambda x, y: (normalization_layer(x), y), 
    #                 num_parallel_calls=tf.data.AUTOTUNE)

    train_ds2 = train_ds2.map(lambda x, y: (normalization_layer(x), y), 
                    num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds2 = val_ds2.map(lambda x, y: (normalization_layer(x), y), 
    #                 num_parallel_calls=tf.data.AUTOTUNE)
    train_ds_X = train_ds.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    train_ds_X1 = train_ds1.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    train_ds_X2 = train_ds2.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    train_ds_Y = train_ds.map(lambda x,y:(y), num_parallel_calls=tf.data.AUTOTUNE)


    # val_ds_X = val_ds.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds_X1 = val_ds1.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds_X2 = val_ds2.map(lambda x,y:(x), num_parallel_calls=tf.data.AUTOTUNE)
    # val_ds_Y = val_ds.map(lambda x,y:(y), num_parallel_calls=tf.data.AUTOTUNE)
    
    # combine left, right dataset 6 combinations
    train_ds_X_X1 = tf.data.Dataset.zip((train_ds_X, train_ds_X1))

    train_ds_X1_X = tf.data.Dataset.zip((train_ds_X1, train_ds_X))

    train_ds_X1_X2 = tf.data.Dataset.zip((train_ds_X1, train_ds_X2))

    train_ds_X2_X1 = tf.data.Dataset.zip((train_ds_X2, train_ds_X1))

    train_ds_X_X2 = tf.data.Dataset.zip((train_ds_X, train_ds_X2))

    train_ds_X2_X = tf.data.Dataset.zip((train_ds_X2, train_ds_X))

    #--------------------------------------------------------------
#     val_ds_X_X1 = tf.data.Dataset.zip((val_ds_X, val_ds_X1))

#     val_ds_X1_X = tf.data.Dataset.zip((val_ds_X1, val_ds_X))

#     val_ds_X1_X2 = tf.data.Dataset.zip((val_ds_X1, val_ds_X2))

#     val_ds_X2_X1 = tf.data.Dataset.zip((val_ds_X2, val_ds_X1))

#     val_ds_X_X2 = tf.data.Dataset.zip((val_ds_X, val_ds_X2))

#     val_ds_X2_X = tf.data.Dataset.zip((val_ds_X2, val_ds_X))
    
    # attach label back
    train_ds_XL_XM_Y = tf.data.Dataset.zip((train_ds_X_X1, train_ds_Y))
    train_ds_XM_XL_Y = tf.data.Dataset.zip((train_ds_X1_X, train_ds_Y))
    train_ds_XM_XR_Y = tf.data.Dataset.zip((train_ds_X1_X2, train_ds_Y))
    train_ds_XR_XM_Y = tf.data.Dataset.zip((train_ds_X2_X1, train_ds_Y))
    train_ds_XL_XR_Y = tf.data.Dataset.zip((train_ds_X_X2, train_ds_Y))
    train_ds_XR_XL_Y = tf.data.Dataset.zip((train_ds_X2_X, train_ds_Y))

    #--------------------------------------------------------------
    # val_ds_XL_XM_Y = tf.data.Dataset.zip((val_ds_X_X1, val_ds_Y))
    # val_ds_XM_XL_Y = tf.data.Dataset.zip((val_ds_X1_X, val_ds_Y))
    # val_ds_XM_XR_Y = tf.data.Dataset.zip((val_ds_X1_X2, val_ds_Y))
    # val_ds_XR_XM_Y = tf.data.Dataset.zip((val_ds_X2_X1, val_ds_Y))
    # val_./models/Model_Early_Combine_all_Vivo-V21_2022-10-22_01-04.h5./models/Model_Early_Combine_all_Vivo-V21_2022-10-22_01-04.h5./models/Model_Early_Combine_all_Vivo-V21_2022-10-22_01-04.h5ds_XL_XR_Y = tf.data.Dataset.zip((val_ds_X_X2, val_ds_Y))
    # val_ds_XR_XL_Y = tf.data.Dataset.zip((val_ds_X2_X, val_ds_Y))

    val_ds_XL_XM_Y = 0
    val_ds_XM_XL_Y = 0
    val_ds_XM_XR_Y = 0
    val_ds_XR_XM_Y = 0
    val_ds_XL_XR_Y = 0
    val_ds_XR_XL_Y = 0
    
    
    return (train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            )

In [7]:
def predictLMR(model, train_ds_XL_XM_Y, val_ds_XL_XM_Y,
            train_ds_XM_XL_Y, val_ds_XM_XL_Y,
            train_ds_XM_XR_Y, val_ds_XM_XR_Y,
            train_ds_XR_XM_Y, val_ds_XR_XM_Y,
            train_ds_XL_XR_Y, val_ds_XL_XR_Y,
            train_ds_XR_XL_Y, val_ds_XR_XL_Y):
    Y_pred = model.predict(train_ds_XL_XM_Y)
    Y_pred = np.hstack((Y_pred, model.predict(train_ds_XM_XL_Y)))
    Y_pred = np.hstack((Y_pred, model.predict(train_ds_XM_XR_Y)))
    Y_pred = np.hstack((Y_pred, model.predict(train_ds_XR_XM_Y)))
    Y_pred = np.hstack((Y_pred, model.predict(train_ds_XL_XR_Y)))
    Y_pred = np.hstack((Y_pred, model.predict(train_ds_XR_XL_Y)))

    # Ypred1 = model.predict(val_ds_XL_XM_Y)
    # Ypred1 = np.hstack((Ypred1, model.predict(val_ds_XM_XL_Y)))
    # Ypred1 = np.hstack((Ypred1, model.predict(val_ds_XM_XR_Y)))
    # Ypred1 = np.hstack((Ypred1, model.predict(val_ds_XR_XM_Y)))
    # Ypred1 = np.hstack((Ypred1, model.predict(val_ds_XL_XR_Y)))
    # Ypred1 = np.hstack((Ypred1, model.predict(val_ds_XR_XL_Y)))
    Ypred1 = 0
    return (Y_pred, Ypred1)

In [8]:
earlyModelPath="./models/Model_Early_Combine_all_Vivo-V21_2022-10-22_01-04.h5"
middleModelPath="./models/Model_Middle_Combine_all_Vivo-V21_2022-10-22_07-50.h5"
lateModelPath="./models/Model_Late_Combine_all_Vivo-V21_2022-10-22_09-23.h5"
ensemblePath="./models/Ensemble_all_Vivo-V21_2022-10-22_12-45.h5"